In [1]:
from linkml_runtime.utils.schemaview import SchemaView
import pandas as pd
import sqlite3
import os

import pickle

# # what/which to import for prsing Montana's google sheet?
# from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import json

# # pip install google-api-python-client google-auth-oauthlib python-igraph

## What slots are *required* for biosamples?! How would I know?

https://microbiomedata.github.io/nmdc-schema/Biosample/

These are marked required

- biosample➞env_broad_scale 1..1
- biosample➞env_local_scale 1..1
- biosample➞env_medium 1..1

`id` has `identifier=True`

This script already for required and recommended MIxS like `elev` and `depth` in the soil package


### DataHarmonizer data types

| datatype              | description                                                                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| xs:token              | An XML string                                                                                                                                      |
| xs:unique             | A xs:token which should be unique in a   dataset. Good for validating sample identifiers for example                                               |
| xs:date               | An XML date                                                                                                                                        |
| select                | A field with a pulldown menu of selection   options. Can be indented as a hierarchy.                                                               |
| multiple              | A field with a popup menu allowing   multiple selection/deselection of items in a given hierarchy of terms.                                        |
| xs:nonNegativeInteger | An integer >= 0                                                                                                                                    |
| xs:decimal            | A decimal number                                                                                                                                   |
| provenance            | Marks a field that when Validated,   automatically receives a prefix of "DataHarmonizer provenance:   vX.Y.Z" in addition to its existing content. |

In [7]:
Example_Soil_NMDC_SampleMetadata = "1GZayIFIrY2jdoxRIpk9KDTBLiE71VVtb7YAd5ZSYGR0"

In [8]:
GOOGLE_API_SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = Example_Soil_NMDC_SampleMetadata
SHEET_TAB_ID = "EcosystemTerms"

In [9]:
# # envo terms are predominant for env broad scale in soil
# # but that won't always be the case
# # where to draw the line between REST API calls which don't require any setup
# # and rdftab databases etc which arent' dependent on remote services

# connection = sqlite3.connect(envo_rdftab_file)

# biomes_q = """
# select s2.subject, s2.value
# from entailed_edge s1
# join statements s2
# on s1.subject = s2.subject
# where s1.predicate = 'rdfs:subClassOf'
# and s1.object = 'ENVO:00000428'
# and s2.predicate = 'rdfs:label'
# ---limit 9
# """

# biomes_res = pd.read_sql_query(biomes_q, connection)

# # biomes_res

---

In [13]:
# https://towardsdatascience.com/how-to-import-google-sheets-data-into-a-pandas-dataframe-using-googles-api-v4-2020-f50e84ea4530


def gsheet_api_check(GOOGLE_API_SCOPES):
    creds = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", GOOGLE_API_SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return creds


def pull_sheet_data(GOOGLE_API_SCOPES, SPREADSHEET_ID, SHEET_TAB_ID):
    creds = gsheet_api_check(GOOGLE_API_SCOPES)
    service = build("sheets", "v4", credentials=creds)
    sheet = service.spreadsheets()
    result = (
        sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=SHEET_TAB_ID).execute()
    )
    values = result.get("values", [])

    if not values:
        print("No data found.")
    else:
        rows = (
            sheet.values()
            .get(spreadsheetId=SPREADSHEET_ID, range=SHEET_TAB_ID)
            .execute()
        )
        data = rows.get("values")
        print("COMPLETE: Data copied")
        return data

In [14]:
pulled_data = pull_sheet_data(GOOGLE_API_SCOPES, SPREADSHEET_ID, SHEET_TAB_ID)

COMPLETE: Data copied


In [15]:
triad_slots = ["env_broad_scale", "env_local_scale", "env_medium"]

et_triad_frame = pd.DataFrame(pulled_data[2:], columns=pulled_data[1])
# there are two sets of package	env_broad_scale	env_local_scale	env_medium columns
# proablay shouldn't assume these locations
et_triad_frame = et_triad_frame.iloc[:, 8:12]
et_triad_frame = et_triad_frame.loc[~et_triad_frame["package"].isnull()]
et_triad_frame = et_triad_frame.loc[
    ~et_triad_frame["env_broad_scale"].eq("select broad scale")
]
et_triad_frame = et_triad_frame.fillna("")
# et_triad_frame

In [16]:
melted_triad = pd.melt(
    et_triad_frame,
    id_vars="package",
    var_name=None,
    value_name="value",
    ignore_index=True,
)

melted_triad = melted_triad.loc[melted_triad["value"].str.len().ge(1)]

----

In [70]:
current_package = "soil"
current_slot = "env_local_scale"

current_triad = melted_triad.loc[
    melted_triad["package"].eq(current_package)
    & melted_triad["variable"].eq(current_slot)
]

current_subgraph_input = list(current_triad["value"])
current_subgraph_input = "' '".join(current_subgraph_input)
current_subgraph_input = "'" + current_subgraph_input + "'"

semantic-sql % 

```
python semsql/subgraph.py \
-d db/envo.db \
-f obojson  \
-p rdfs:subClassOf \
-V self  \
-m label 
```

In [71]:
print(current_subgraph_input)

'active geological fault' 'agricultural field' 'beach' 'canyon' 'cave' 'vein' 'channel' 'cliff' 'coast' 'dry lake' 'dry river' 'dry valley' 'dune' 'garden' 'glacial valley' 'hill' 'hillside' 'hummock' 'impact crater' 'isthmus' 'karst' 'lake shore' 'lava field' 'mesa' 'mountain' 'peninsula' 'plain' 'plateau' 'ridge' 'slope' 'snow field' 'tombolo' 'tuff cone' 'tunnel' 'valley' 'volcano' 'woodland clearing'


```
semantic-sql % python semsql/subgraph.py \
-d db/envo.db \
-f obojson  \
-p rdfs:subClassOf \
-V self  \
-m label 'alpine biome' 'alpine tundra biome' 'anthropogenic terrestrial biome' 'arid biome' 'flooded savanna biome' 'mangrove biome' 'mediterranean biome' 'mediterranean savanna biome' 'mediterranean shrubland biome' 'mediterranean woodland biome' 'montane biome' 'montane savanna biome' 'montane shrubland biome' 'polar biome' 'rangeland biome' 'savanna biome' 'shrubland biome' 'subalpine biome' 'subpolar biome' 'subtropical biome' 'subtropical savanna biome' 'subtropical shrubland biome' 'subtropical woodland biome' 'temperate biome' 'temperate savanna biome' 'temperate shrubland biome' 'temperate woodland biome' 'tropical biome' 'tropical savanna biome' 'tropical shrubland biome' 'tropical woodland biome' 'tundra biome' 'urban biome' 'village biome' 'woodland biome' > soil_ebs.obo.json
```

In [72]:
f = open("../../semantic-sql/current_extract.obo.json")

current_extract = json.load(f)
f.close()
current_edges = pd.DataFrame(current_extract["edges"])
# current_edges

In [73]:
current_label_dicts = [
    {"id": i["id"], "label": i["lbl"]} for i in current_extract["nodes"]
]
current_label_frame = pd.DataFrame(current_label_dicts)
# current_label_frame

In [74]:
# current_edges = pd.mer

for_dh = current_edges.merge(
    current_label_frame,
    how="inner",
    left_on="sub",
    right_on="id",
    sort=False,
    copy=True,
    indicator=False,
    validate=None,
)

for_dh = for_dh.merge(
    current_label_frame,
    how="inner",
    left_on="obj",
    right_on="id",
    sort=False,
    copy=True,
    indicator=False,
    validate=None,
)

for_dh = for_dh[["id_x", "label_x", "label_y"]]
for_dh.columns = ["child_id", "child", "parent"]
for_dh.sort_values(by=["parent", "child"], inplace=True)
for_dh.to_csv("current_for_review.tsv", sep="\t", index=False)